# 马尔可夫决策过程与动态规划

**Markov Decision Process & Dynamic Programming**

---

## 学习目标

完成本教程后，你将掌握：

1. **MDP 数学框架**: 状态、动作、转移、奖励、折扣因子
2. **贝尔曼方程**: 期望方程与最优方程的推导与直觉
3. **动态规划算法**: 策略评估、策略改进、策略迭代、值迭代
4. **实践技能**: 在 GridWorld 环境中实现和验证算法

## 前置知识

- Python 编程基础
- NumPy 数组操作
- 概率论基础（条件概率、期望）
- 基础线性代数

## 预计时间

**60-90 分钟** (可分多次完成)

---

## 目录

1. [理论基础](#part1)
   - 1.1 强化学习框架
   - 1.2 MDP 形式化定义
   - 1.3 策略与价值函数
   - 1.4 贝尔曼方程

2. [环境实现](#part2)
   - 2.1 库导入与配置
   - 2.2 GridWorld 环境
   - 2.3 环境可视化

3. [动态规划算法](#part3)
   - 3.1 策略评估
   - 3.2 策略改进
   - 3.3 策略迭代
   - 3.4 值迭代

4. [结果分析](#part4)
   - 4.1 算法对比
   - 4.2 策略执行
   - 4.3 参数实验

5. [总结与练习](#part5)

---

<a id="part1"></a>
# Part 1: 理论基础

## 1.1 强化学习框架

强化学习研究**智能体 (Agent)** 如何在与**环境 (Environment)** 的交互中学习最优策略。

```
                    动作 aₜ
         ┌────────────────────┐
         │                    ▼
    ┌─────────┐          ┌─────────┐
    │  Agent  │          │   Env   │
    │ (智能体) │          │  (环境)  │
    └─────────┘          └─────────┘
         ▲                    │
         │   状态 sₜ₊₁        │
         │   奖励 rₜ₊₁        │
         └────────────────────┘
```

**交互循环**：
1. 智能体观测当前状态 $s_t$
2. 根据策略选择动作 $a_t$
3. 环境转移至新状态 $s_{t+1}$，反馈奖励 $r_{t+1}$
4. 智能体利用反馈更新策略

## 1.2 MDP 形式化定义

马尔可夫决策过程由**五元组**定义：

$$\mathcal{M} = \langle \mathcal{S}, \mathcal{A}, P, R, \gamma \rangle$$

| 符号 | 名称 | 定义 |
|:----:|:----:|:----|
| $\mathcal{S}$ | 状态空间 | 所有可能状态的集合 |
| $\mathcal{A}$ | 动作空间 | 所有可能动作的集合 |
| $P$ | 转移函数 | $P(s' \mid s, a) = \Pr(S_{t+1}=s' \mid S_t=s, A_t=a)$ |
| $R$ | 奖励函数 | $R(s, a, s')$ 或简化为 $R(s, a)$ |
| $\gamma$ | 折扣因子 | $\gamma \in [0, 1]$，控制对未来奖励的重视程度 |

### 马尔可夫性质

> **核心假设**：未来状态仅依赖当前状态，与历史无关

$$P(S_{t+1} \mid S_t, A_t, S_{t-1}, A_{t-1}, \ldots) = P(S_{t+1} \mid S_t, A_t)$$

**直觉理解**：当前状态包含了预测未来所需的全部信息。这一性质使得动态规划方法成为可能。

## 1.3 策略与价值函数

### 策略 (Policy)

策略 $\pi$ 定义智能体在各状态下的行为方式：

- **随机策略**: $\pi(a \mid s) = \Pr(A_t = a \mid S_t = s)$
- **确定性策略**: $a = \pi(s)$

### 状态价值函数 $V^\pi(s)$

从状态 $s$ 出发，遵循策略 $\pi$ 的期望累积回报：

$$V^\pi(s) = \mathbb{E}_\pi\left[\sum_{k=0}^{\infty} \gamma^k R_{t+k+1} \mid S_t = s\right]$$

### 动作价值函数 $Q^\pi(s, a)$

从状态 $s$ 执行动作 $a$，然后遵循策略 $\pi$：

$$Q^\pi(s, a) = \mathbb{E}_\pi\left[\sum_{k=0}^{\infty} \gamma^k R_{t+k+1} \mid S_t = s, A_t = a\right]$$

## 1.4 贝尔曼方程

### 贝尔曼期望方程

给定策略 $\pi$ 的价值函数满足递归关系：

$$V^\pi(s) = \sum_{a} \pi(a \mid s) \left[ R(s, a) + \gamma \sum_{s'} P(s' \mid s, a) V^\pi(s') \right]$$

**直觉**: 当前价值 = 即时奖励期望 + 折扣后的后继状态价值期望

### 贝尔曼最优方程

最优价值函数满足：

$$V^*(s) = \max_a \left[ R(s, a) + \gamma \sum_{s'} P(s' \mid s, a) V^*(s') \right]$$

**直觉**: 最优价值 = 选择最优动作后能获得的最大期望回报

---

<a id="part2"></a>
# Part 2: 环境实现

## 2.1 库导入与配置

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, field

# 设置随机种子保证可复现性
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
# Matplotlib 中文显示配置
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

print("环境配置完成")

## 2.2 GridWorld 环境

### 环境描述

GridWorld 是强化学习的经典测试环境：

- **目标**: 智能体从起点导航到目标点
- **动作**: 上、下、左、右 四个方向移动
- **奖励**: 每步 -1（鼓励快速到达），到达目标 +100
- **障碍**: 无法通过的格子，碰到会停留原地

```
┌────┬────┬────┬────┐
│ S  │    │    │    │   S: 起点 (0,0)
├────┼────┼────┼────┤   G: 目标 (3,3)
│    │ X  │    │    │   X: 障碍物
├────┼────┼────┼────┤
│    │    │    │    │
├────┼────┼────┼────┤
│    │    │    │ G  │
└────┴────┴────┴────┘
```

In [ ]:
@dataclass
class GridConfig:
    """网格世界配置参数"""
    size: int = 4
    start: Tuple[int, int] = (0, 0)
    goal: Tuple[int, int] = (3, 3)
    obstacles: List[Tuple[int, int]] = field(default_factory=list)
    step_reward: float = -1.0
    goal_reward: float = 100.0
    slip_prob: float = 0.0
    
    def __post_init__(self):
        if not self.obstacles:
            self.obstacles = [(1, 1)]

In [ ]:
# 创建默认配置
config = GridConfig()

print("GridWorld 配置:")
print(f"  网格大小: {config.size}×{config.size}")
print(f"  起点: {config.start}")
print(f"  目标: {config.goal}")
print(f"  障碍物: {config.obstacles}")
print(f"  每步奖励: {config.step_reward}")
print(f"  目标奖励: {config.goal_reward}")

In [ ]:
class GridWorld:
    """
    网格世界 MDP 环境
    
    状态: (行, 列) 元组
    动作: '上', '下', '左', '右'
    """
    
    ACTION_DELTAS = {
        '上': (-1, 0),
        '下': (1, 0),
        '左': (0, -1),
        '右': (0, 1)
    }
    
    def __init__(self, config: GridConfig):
        self.config = config
        self.size = config.size
        self.goal = config.goal
        self.obstacles = set(config.obstacles)
        
        # 构建状态空间
        self.states = [
            (i, j) for i in range(self.size) 
            for j in range(self.size)
            if (i, j) not in self.obstacles
        ]
        
        self.actions = list(self.ACTION_DELTAS.keys())

In [ ]:
# 添加环境核心方法
def is_terminal(self, state):
    """判断是否为终止状态"""
    return state == self.goal

def get_next_state(self, state, action):
    """计算下一状态"""
    if self.is_terminal(state):
        return state
        
    di, dj = self.ACTION_DELTAS[action]
    ni = max(0, min(self.size - 1, state[0] + di))
    nj = max(0, min(self.size - 1, state[1] + dj))
    next_state = (ni, nj)
    
    if next_state in self.obstacles:
        return state
    return next_state

def get_reward(self, state, next_state):
    """获取奖励"""
    if next_state == self.goal:
        return self.config.goal_reward
    return self.config.step_reward

def get_transitions(self, state, action):
    """获取状态转移分布"""
    if self.is_terminal(state):
        return [(state, 1.0, 0.0)]
    
    next_state = self.get_next_state(state, action)
    reward = self.get_reward(state, next_state)
    return [(next_state, 1.0, reward)]

# 绑定方法到类
GridWorld.is_terminal = is_terminal
GridWorld.get_next_state = get_next_state
GridWorld.get_reward = get_reward
GridWorld.get_transitions = get_transitions

In [ ]:
# 创建环境实例
env = GridWorld(config)

print("环境创建成功")
print(f"  状态空间大小: {len(env.states)}")
print(f"  动作空间: {env.actions}")

## 2.3 环境可视化

In [ ]:
def visualize_grid(env, title="GridWorld 环境"):
    """可视化网格世界"""
    fig, ax = plt.subplots(figsize=(6, 6))
    
    # 绘制网格线
    for i in range(env.size + 1):
        ax.axhline(y=i, color='black', linewidth=1)
        ax.axvline(x=i, color='black', linewidth=1)
    
    # 标记特殊位置
    for i in range(env.size):
        for j in range(env.size):
            y = env.size - i - 0.5
            x = j + 0.5
            
            if (i, j) == env.config.start:
                ax.text(x, y, 'S', ha='center', va='center', 
                       fontsize=20, fontweight='bold', color='blue')
            elif (i, j) == env.goal:
                ax.add_patch(plt.Rectangle((j, env.size-i-1), 1, 1, 
                            facecolor='lightgreen', alpha=0.5))
                ax.text(x, y, 'G', ha='center', va='center', 
                       fontsize=20, fontweight='bold', color='green')
            elif (i, j) in env.obstacles:
                ax.add_patch(plt.Rectangle((j, env.size-i-1), 1, 1, 
                            facecolor='gray', alpha=0.8))
                ax.text(x, y, 'X', ha='center', va='center', 
                       fontsize=20, fontweight='bold', color='white')
    
    ax.set_xlim(0, env.size)
    ax.set_ylim(0, env.size)
    ax.set_aspect('equal')
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_grid(env)

---

<a id="part3"></a>
# Part 3: 动态规划算法

## 3.1 策略评估 (Policy Evaluation)

### 算法思想

给定策略 $\pi$，通过迭代计算其价值函数 $V^\pi$。

**更新规则**:

$$V_{k+1}(s) = \sum_{a} \pi(a \mid s) \sum_{s'} P(s' \mid s, a) [R(s,a,s') + \gamma V_k(s')]$$

### 为什么这样做？

1. **不动点迭代**: 贝尔曼方程定义了 $V^\pi$ 为不动点
2. **收缩映射**: 贝尔曼算子是 $\gamma$-收缩，保证收敛
3. **收敛速度**: 线性收敛，误差以 $\gamma^k$ 速率衰减

In [ ]:
def policy_evaluation(env, policy, gamma=0.99, theta=1e-6, max_iters=10000):
    """
    策略评估：计算给定策略的状态价值函数
    
    Args:
        env: 环境
        policy: 策略 π(a|s)
        gamma: 折扣因子
        theta: 收敛阈值
        max_iters: 最大迭代次数
        
    Returns:
        (V, iterations): 价值函数和迭代次数
    """
    V = {s: 0.0 for s in env.states}
    
    for iteration in range(1, max_iters + 1):
        delta = 0.0
        
        for state in env.states:
            if env.is_terminal(state):
                continue
                
            old_value = V[state]
            new_value = 0.0
            
            # 贝尔曼期望方程
            for action in env.actions:
                action_prob = policy.get(state, {}).get(action, 0.0)
                if action_prob > 0:
                    for ns, tp, r in env.get_transitions(state, action):
                        new_value += action_prob * tp * (r + gamma * V.get(ns, 0.0))
            
            V[state] = new_value
            delta = max(delta, abs(old_value - new_value))
        
        if delta < theta:
            return V, iteration
    
    return V, max_iters

In [ ]:
# 创建均匀随机策略
random_policy = {
    s: {a: 0.25 for a in env.actions}
    for s in env.states
}

print("随机策略示例 (每个动作概率相等):")
print(f"状态 (0,0): {random_policy[(0, 0)]}")

In [ ]:
# 评估随机策略
V_random, iters = policy_evaluation(env, random_policy, gamma=0.99)

print(f"策略评估完成，迭代次数: {iters}")
print("\n状态价值函数 V^π(s):")
for i in range(env.size):
    row = []
    for j in range(env.size):
        if (i, j) in V_random:
            row.append(f"{V_random[(i, j)]:7.2f}")
        else:
            row.append("   X   ")
    print(" | ".join(row))

## 3.2 策略改进 (Policy Improvement)

### 算法思想

基于当前价值函数，贪婪选择最优动作：

$$\pi'(s) = \arg\max_a \sum_{s'} P(s' \mid s, a) [R(s,a,s') + \gamma V(s')]$$

### 策略改进定理

如果 $\pi'$ 是关于 $V^\pi$ 的贪婪策略，则 $V^{\pi'}(s) \geq V^\pi(s)$ 对所有状态成立。

**直觉**: 选择当前看起来最好的动作，不会使策略变差。

In [ ]:
def policy_improvement(env, V, gamma=0.99):
    """
    策略改进：基于价值函数构造贪婪策略
    
    Args:
        env: 环境
        V: 状态价值函数
        gamma: 折扣因子
        
    Returns:
        改进后的确定性策略
    """
    policy = {}
    
    for state in env.states:
        if env.is_terminal(state):
            policy[state] = {a: 0.25 for a in env.actions}
            continue
        
        # 计算各动作的 Q 值
        q_values = {}
        for action in env.actions:
            q_val = 0.0
            for ns, tp, r in env.get_transitions(state, action):
                q_val += tp * (r + gamma * V.get(ns, 0.0))
            q_values[action] = q_val
        
        # 选择最优动作
        best_value = max(q_values.values())
        best_actions = [a for a, v in q_values.items() 
                       if abs(v - best_value) < 1e-9]
        
        policy[state] = {
            a: 1.0 if a == best_actions[0] else 0.0
            for a in env.actions
        }
    
    return policy

## 3.3 策略迭代 (Policy Iteration)

### 算法流程

```
π₀ → V^π₀ → π₁ → V^π₁ → π₂ → ... → π* → V*
```

1. 初始化随机策略 $\pi_0$
2. **策略评估**: 计算 $V^{\pi_k}$
3. **策略改进**: $\pi_{k+1} = \text{greedy}(V^{\pi_k})$
4. 若 $\pi_{k+1} = \pi_k$ 则停止，否则 $k \leftarrow k+1$ 转步骤 2

### 算法特点

| 优点 | 缺点 |
|:----:|:----:|
| 外层迭代少（3-10次） | 每次评估代价高 |
| 理论收敛保证 | 需要完整模型 |
| 中间策略可执行 | 大状态空间开销大 |

In [ ]:
def policy_iteration(env, gamma=0.99, max_iters=100, verbose=True):
    """
    策略迭代算法
    
    Args:
        env: 环境
        gamma: 折扣因子
        max_iters: 最大迭代次数
        verbose: 是否打印进度
        
    Returns:
        (policy, V, iterations)
    """
    # 初始化均匀随机策略
    policy = {s: {a: 0.25 for a in env.actions} for s in env.states}
    V = {}
    
    for iteration in range(1, max_iters + 1):
        # 策略评估
        V, eval_iters = policy_evaluation(env, policy, gamma)
        
        if verbose:
            print(f"迭代 {iteration}: 策略评估用了 {eval_iters} 次内层迭代")
        
        # 策略改进
        new_policy = policy_improvement(env, V, gamma)
        
        # 检查收敛
        if policy == new_policy:
            if verbose:
                print(f"\n策略迭代收敛，总迭代: {iteration}")
            return new_policy, V, iteration
        
        policy = new_policy
    
    return policy, V, max_iters

In [ ]:
print("=" * 50)
print("运行策略迭代")
print("=" * 50)

policy_pi, V_pi, iters_pi = policy_iteration(env, gamma=0.99)

## 3.4 值迭代 (Value Iteration)

### 算法思想

直接迭代贝尔曼最优方程，不显式维护策略：

$$V_{k+1}(s) = \max_a \sum_{s'} P(s' \mid s, a) [R(s,a,s') + \gamma V_k(s')]$$

### 与策略迭代的对比

| 维度 | 策略迭代 | 值迭代 |
|:----:|:--------:|:------:|
| 每次迭代 | 完全策略评估 | 单步贝尔曼更新 |
| 迭代次数 | 少（3-10次） | 多（数百次） |
| 每次代价 | 高 | 低 |
| 中间结果 | 有可执行策略 | 无策略 |

In [ ]:
def value_iteration(env, gamma=0.99, theta=1e-6, max_iters=10000, verbose=True):
    """
    值迭代算法
    
    Args:
        env: 环境
        gamma: 折扣因子
        theta: 收敛阈值
        max_iters: 最大迭代次数
        verbose: 是否打印进度
        
    Returns:
        (policy, V, iterations)
    """
    V = {s: 0.0 for s in env.states}
    
    for iteration in range(1, max_iters + 1):
        delta = 0.0
        
        for state in env.states:
            if env.is_terminal(state):
                continue
            
            old_value = V[state]
            
            # 贝尔曼最优方程
            q_values = []
            for action in env.actions:
                q_val = 0.0
                for ns, tp, r in env.get_transitions(state, action):
                    q_val += tp * (r + gamma * V.get(ns, 0.0))
                q_values.append(q_val)
            
            V[state] = max(q_values)
            delta = max(delta, abs(old_value - V[state]))
        
        if delta < theta:
            if verbose:
                print(f"值迭代收敛，迭代次数: {iteration}")
            policy = policy_improvement(env, V, gamma)
            return policy, V, iteration
    
    policy = policy_improvement(env, V, gamma)
    return policy, V, max_iters

In [ ]:
print("\n" + "=" * 50)
print("运行值迭代")
print("=" * 50)

policy_vi, V_vi, iters_vi = value_iteration(env, gamma=0.99)

---

<a id="part4"></a>
# Part 4: 结果分析

## 4.1 可视化策略和价值函数

In [ ]:
def visualize_policy_values(env, policy, V, title):
    """可视化策略和价值函数"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    arrows = {'上': '↑', '下': '↓', '左': '←', '右': '→'}
    
    # 左图：策略
    ax = axes[0]
    for i in range(env.size + 1):
        ax.axhline(y=i, color='black', linewidth=1)
        ax.axvline(x=i, color='black', linewidth=1)
    
    for i in range(env.size):
        for j in range(env.size):
            y = env.size - i - 0.5
            x = j + 0.5
            
            if (i, j) == env.goal:
                ax.add_patch(plt.Rectangle((j, env.size-i-1), 1, 1, 
                            facecolor='lightgreen', alpha=0.5))
                ax.text(x, y, 'G', ha='center', va='center', 
                       fontsize=16, fontweight='bold', color='green')
            elif (i, j) in env.obstacles:
                ax.add_patch(plt.Rectangle((j, env.size-i-1), 1, 1, 
                            facecolor='gray', alpha=0.8))
                ax.text(x, y, 'X', ha='center', va='center', 
                       fontsize=16, fontweight='bold', color='white')
            elif (i, j) in policy:
                best_action = max(policy[(i, j)], key=policy[(i, j)].get)
                ax.text(x, y, arrows[best_action], ha='center', va='center', 
                       fontsize=20, color='blue')
    
    ax.set_xlim(0, env.size)
    ax.set_ylim(0, env.size)
    ax.set_aspect('equal')
    ax.set_title('最优策略 π*', fontsize=14, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    
    # 右图：价值函数
    ax = axes[1]
    value_matrix = np.zeros((env.size, env.size))
    for i in range(env.size):
        for j in range(env.size):
            value_matrix[i, j] = V.get((i, j), np.nan)
    
    im = ax.imshow(value_matrix, cmap='RdYlGn', aspect='equal')
    
    for i in range(env.size):
        for j in range(env.size):
            if (i, j) == env.goal:
                ax.text(j, i, 'G', ha='center', va='center', 
                       fontsize=14, fontweight='bold')
            elif (i, j) in env.obstacles:
                ax.text(j, i, 'X', ha='center', va='center', 
                       fontsize=14, fontweight='bold', color='white')
            elif (i, j) in V:
                ax.text(j, i, f'{V[(i, j)]:.1f}', ha='center', va='center', 
                       fontsize=10)
    
    ax.set_title('状态价值函数 V*', fontsize=14, fontweight='bold')
    ax.set_xticks([])
    ax.set_yticks([])
    plt.colorbar(im, ax=ax, shrink=0.8)
    
    fig.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_policy_values(env, policy_pi, V_pi, "策略迭代结果")

In [ ]:
visualize_policy_values(env, policy_vi, V_vi, "值迭代结果")

## 4.2 算法对比

In [ ]:
print("=" * 60)
print("策略迭代 vs 值迭代")
print("=" * 60)
print(f"{'算法':<20} {'迭代次数':>15} {'特点':>20}")
print("-" * 60)
print(f"{'策略迭代':<20} {iters_pi:>15} {'精确评估，迭代少':>20}")
print(f"{'值迭代':<20} {iters_vi:>15} {'截断评估，每次代价低':>20}")
print("-" * 60)

# 验证结果一致性
value_diff = sum(abs(V_pi[s] - V_vi[s]) for s in env.states)
print(f"\n价值函数总差异: {value_diff:.2e}")

if value_diff < 1e-5:
    print("两种算法收敛到相同的最优解！")

## 4.3 策略执行

In [ ]:
def execute_policy(env, policy, max_steps=50):
    """执行策略并返回轨迹"""
    state = env.config.start
    trajectory = [state]
    total_reward = 0.0
    
    for step in range(max_steps):
        if env.is_terminal(state):
            break
        
        best_action = max(policy[state], key=policy[state].get)
        transitions = env.get_transitions(state, best_action)
        next_state, _, reward = transitions[0]
        
        total_reward += reward
        state = next_state
        trajectory.append(state)
    
    return total_reward, trajectory

In [ ]:
reward, trajectory = execute_policy(env, policy_vi)

print("执行最优策略:")
print(f"  总奖励: {reward:.1f}")
print(f"  步数: {len(trajectory) - 1}")
print(f"  轨迹: {' → '.join([str(s) for s in trajectory])}")

In [ ]:
def visualize_trajectory(env, trajectory):
    """可视化执行轨迹"""
    fig, ax = plt.subplots(figsize=(7, 7))
    
    for i in range(env.size + 1):
        ax.axhline(y=i, color='black', linewidth=1)
        ax.axvline(x=i, color='black', linewidth=1)
    
    for i in range(env.size):
        for j in range(env.size):
            if (i, j) == env.goal:
                ax.add_patch(plt.Rectangle((j, env.size-i-1), 1, 1, 
                            facecolor='lightgreen', alpha=0.5))
            elif (i, j) in env.obstacles:
                ax.add_patch(plt.Rectangle((j, env.size-i-1), 1, 1, 
                            facecolor='gray', alpha=0.8))
    
    # 绘制轨迹
    for idx, (i, j) in enumerate(trajectory):
        y = env.size - i - 0.5
        x = j + 0.5
        
        if idx == 0:
            ax.plot(x, y, 'bo', markersize=15, label='起点')
        elif idx == len(trajectory) - 1:
            ax.plot(x, y, 'g*', markersize=20, label='终点')
        else:
            ax.plot(x, y, 'r.', markersize=10)
    
    xs = [t[1] + 0.5 for t in trajectory]
    ys = [env.size - t[0] - 0.5 for t in trajectory]
    ax.plot(xs, ys, 'r-', linewidth=2, alpha=0.7, label='轨迹')
    
    ax.set_xlim(0, env.size)
    ax.set_ylim(0, env.size)
    ax.set_aspect('equal')
    ax.set_title('最优策略执行轨迹', fontsize=14, fontweight='bold')
    ax.legend(loc='upper right')
    ax.set_xticks([])
    ax.set_yticks([])
    plt.tight_layout()
    plt.show()

visualize_trajectory(env, trajectory)

## 4.4 参数实验：折扣因子 γ

In [ ]:
gammas = [0.5, 0.9, 0.99]

print("不同折扣因子 γ 对价值函数的影响:")
print("=" * 60)

for gamma in gammas:
    _, V_gamma, iters = value_iteration(env, gamma=gamma, verbose=False)
    
    print(f"\nγ = {gamma}, 收敛迭代: {iters}")
    print(f"  起点 V(0,0) = {V_gamma[(0, 0)]:.2f}")
    print(f"  目标邻近 V(3,2) = {V_gamma[(3, 2)]:.2f}")

print("\n" + "=" * 60)
print("观察：γ 越大，远离目标的状态价值越高（更重视长期回报）")

---

<a id="part5"></a>
# Part 5: 总结与练习

## 核心知识点回顾

### MDP 五元组

$$\mathcal{M} = \langle \mathcal{S}, \mathcal{A}, P, R, \gamma \rangle$$

### 贝尔曼方程

| 类型 | 方程 | 用途 |
|:----:|:----:|:----:|
| 期望方程 | $V^\pi(s) = \sum_a \pi(a|s)[R + \gamma \sum_{s'} P V^\pi(s')]$ | 策略评估 |
| 最优方程 | $V^*(s) = \max_a [R + \gamma \sum_{s'} P V^*(s')]$ | 值迭代 |

### 算法对比

| 维度 | 策略迭代 | 值迭代 |
|:----:|:--------:|:------:|
| 外层迭代 | 少 | 多 |
| 内层代价 | 高 | 低 |
| 中间策略 | 有 | 无 |
| 适用场景 | 小状态空间 | 大状态空间 |

## 单元测试

In [ ]:
def run_tests():
    """运行单元测试"""
    print("开始单元测试...\n")
    passed = 0
    failed = 0
    
    # 测试1: 状态空间
    try:
        assert len(env.states) == 15
        assert (1, 1) not in env.states
        print("测试1通过: 状态空间正确")
        passed += 1
    except AssertionError as e:
        print(f"测试1失败: {e}")
        failed += 1
    
    # 测试2: 终止状态
    try:
        assert env.is_terminal((3, 3)) == True
        assert env.is_terminal((0, 0)) == False
        print("测试2通过: 终止状态判断正确")
        passed += 1
    except AssertionError as e:
        print(f"测试2失败: {e}")
        failed += 1
    
    # 测试3: 状态转移
    try:
        next_s = env.get_next_state((0, 0), '下')
        assert next_s == (1, 0)
        next_s = env.get_next_state((0, 0), '上')
        assert next_s == (0, 0)  # 边界
        print("测试3通过: 状态转移正确")
        passed += 1
    except AssertionError as e:
        print(f"测试3失败: {e}")
        failed += 1
    
    # 测试4: 策略迭代收敛
    try:
        assert iters_pi <= 10
        print(f"测试4通过: 策略迭代在 {iters_pi} 次收敛")
        passed += 1
    except AssertionError as e:
        print(f"测试4失败: {e}")
        failed += 1
    
    # 测试5: 结果一致性
    try:
        diff = sum(abs(V_pi[s] - V_vi[s]) for s in env.states)
        assert diff < 1e-5
        print("测试5通过: 两种算法结果一致")
        passed += 1
    except AssertionError as e:
        print(f"测试5失败: {e}")
        failed += 1
    
    # 测试6: 最优策略到达目标
    try:
        reward, traj = execute_policy(env, policy_vi)
        assert traj[-1] == env.goal
        print(f"测试6通过: 最优策略在 {len(traj)-1} 步到达目标")
        passed += 1
    except AssertionError as e:
        print(f"测试6失败: {e}")
        failed += 1
    
    print(f"\n{'='*50}")
    print(f"测试完成: {passed} 通过, {failed} 失败")
    return failed == 0

run_tests()

## 练习题

### 练习 1: 修改环境

创建一个带有更多障碍物的 5×5 网格，观察最优策略的变化。

### 练习 2: 随机环境

修改 `get_transitions` 方法实现随机转移（滑动概率 20%），比较确定性和随机环境下的策略差异。

### 练习 3: 收敛分析

绘制不同 γ 值下值迭代的收敛曲线（迭代次数 vs 最大价值变化）。

### 练习 4: Q-Learning 预热

实现从 V 计算 Q 的函数，为下一章 Q-Learning 做准备：

$$Q(s,a) = R(s,a) + \gamma \sum_{s'} P(s'|s,a) V(s')$$

---

## 参考资料

1. Sutton & Barto, *Reinforcement Learning: An Introduction* (2nd ed.), Chapter 4
2. Bellman, R. *Dynamic Programming*, Princeton University Press, 1957
3. Howard, R. *Dynamic Programming and Markov Processes*, MIT Press, 1960
4. David Silver, UCL RL Course, Lectures 2-3